In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df1_WS4T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_OF_segment1_workflowS4_has_duration.csv",engine='python')

In [3]:
df1_WS4T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1140679|518493|795102,Release Offer,Release Offer,25/10/2019 11:48:16,KRIT,Faculty,4,N,3b. Offer stage,Part 2 - Application Line ID Workflow,...,MCW,HDCW,Full Degree,OF,OF,Y,NaN,Y,0 days 0 hours 0 minutes 37 seconds,37.0


In [4]:
numOfWorkflowProcesses = df1_WS4T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df1_WS4T[df1_WS4T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Accept','Defer'])]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [11]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step)

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [12]:
s1_t4=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s1_t4['Num of Occurrences per step(descending)'] = list_step_count_per_key
s1_t4['Average time spends per step(descending)'] = list_step_ave_time_per_key
s1_t4.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step,Num of Occurrences per step(descending),Average time spends per step(descending)
0,1,14,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Release Offer},"[(Release Offer, 14)]",[]
1,2,25,35 days 10 hours 42 minutes 9 seconds,13.0 days 5.0 hours 57.0 minutes 15.0 seconds,72.0%,"{Defer, Assessment Outcome - Successful, AR - ...","[(Release Offer, 25), (Accept, 13), (Defer, 5)...","[(Release Offer, 35.0 days 10.0 hours 42.0 min..."
2,3,19,44 days 22 hours 24 minutes 52 seconds,23.0 days 0.0 hours 42.0 minutes 5.0 seconds,78.95%,"{Send for Assessment, Defer, AR - Reviewed - f...","[(Release Offer, 19), (Accept, 11), (Defer, 4)...","[(Release Offer, 39.0 days 6.0 hours 22.0 minu..."
3,4,6,67 days 10 hours 33 minutes 33 seconds,44.0 days 12.0 hours 28.0 minutes 54.0 seconds,83.33%,"{AR - Satisfied - Other, AR - Satisfied - ELR,...","[(Release Offer, 6), (Accept, 5), (OC - Satisf...","[(OC - Satisfied - Other, 6.0 days 6.0 hours 4..."
4,5,2,75 days 17 hours 37 minutes 45 seconds,75.0 days 17.0 hours 37.0 minutes 45.5 seconds,100.0%,"{End My Assessment, AR - Satisfied - work expe...","[(Release Offer, 2), (Accept, 2), (OC - Satisf...","[(OC - Created - Other, 90.0 days 7.0 hours 26..."
5,6,1,229 days 5 hours 5 minutes 23 seconds,229.0 days 5.0 hours 5.0 minutes 23.0 seconds,100.0%,"{AR - Satisfied - Other, AR - Created - Other,...","[(AR - Satisfied - Other, 2), (Release Offer, ...","[(Release Offer, 39.0 days 23.0 hours 36.0 min..."
6,7,6,77 days 23 hours 42 minutes 20 seconds,55.0 days 10.0 hours 55.0 minutes 5.5 seconds,100.0%,"{AR - Satisfied - Other, Acacdemic - Assessmen...","[(OC - Satisfied - Other, 10), (Release Offer,...","[(AR - Satisfied - Other, 7.0 days 18.0 hours ..."
7,8,4,38 days 7 hours 4 minutes 50 seconds,37.0 days 12.0 hours 2.0 minutes 53.0 seconds,100.0%,"{AR - Satisfied - Other, AR - Satisfied - ELR,...","[(AR - Satisfied - Other, 5), (Release Offer, ...","[(AR - Satisfied - advanced standing, 9.0 days..."


In [13]:
s1_t4.to_csv('engineering_OF_segment1_workflowS4_table.csv', index=False)





